In [10]:
from clustering.clustering import get_clusterer
from data.read_data import *
from dimensionality_reduction.embedding import get_embedder
from experiment import Experiment

import argparse
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

Clustering results with infinite gaussian with dirichlet process prior

In [11]:
twosides = get_twosides_meddra(pickle=False)
bayesian_results = pd.read_csv("../results/bayesian_gaussian/results.csv")
#kmeans_meddra = match_meddra(bayesian_results, twosides)

#(kmeans_meddra.groupby(["cluster", "soc_term"]).size()/kmeans_meddra.groupby(["cluster"]).size())\
#.sort_values(ascending=False)\
#.sort_index(level=0, sort_remaining=False)\
#.groupby("cluster").nlargest(8)

<img src="../results/bayesian_gaussian/embedded_clusters.png">

In [12]:
twosides = get_twosides_meddra(pickle=False)
kmeans10 = pd.read_csv("../results/kmeans_10/kmeans_10_results.csv")

#(kmeans_meddra.groupby(["cluster", "soc_term"]).size()/kmeans_meddra.groupby(["cluster"]).size())\
#.sort_values(ascending=False)\
#.sort_index(level=0, sort_remaining=False)\
#.groupby("cluster").nlargest(8)

In [13]:
def intersect2D(a, b):
  """
  Find row intersection between 2D numpy arrays, a and b.
  Returns another numpy array with shared rows
  """
  return np.array([x for x in set(tuple(x) for x in a) & set(tuple(x) for x in b)])

In [62]:
def swap(a, i, j, r):
    """
    Takes a numpy array and swaps matching elements
    """
    if r == "col":
        copy = np.copy(a[:, i])
        a[:, i] = a[:, j]
        a[:, j] = copy
    elif r == "row":
        copy = np.copy(a[i, :])
        a[i, :] = a[j, :]
        a[j, :] = copy
    return a

In [450]:
def max_swap(a):
    for i in range(min(a.shape)):
        sub = a[i:, i:]
        maxi = np.argmax(sub)
        ind1 = i + (maxi // (a.shape[1] - i)) % a.shape[0]
        ind2 = (maxi % (a.shape[1])) + i
        print(sub, i)
        print(ind1, ind2)
        cpy2 = np.copy(a[:, i])
        a[:, i] = a[:, ind2]
        a[:, ind2] = cpy2
        cpy1 = np.copy(a[i, :])
        #print(cpy1, cpy2)
        a[i, :] = a[ind1, :]
        a[ind1, :] = cpy1
    return a

In [451]:
a = np.array([[10, 1, 2, 9, 2],[3, 4, 5, 11, 2], [6, 7, 8, 12, 15]])
maxi = np.argmax(a)
maxi % a.shape[1]
(maxi // a.shape[1]) % a.shape[0]

2

In [452]:
a.shape
max_swap(a)

[[10  1  2  9  2]
 [ 3  4  5 11  2]
 [ 6  7  8 12 15]] 0
2 4
[[ 4  5 11  3]
 [ 1  2  9 10]] 1
1 3
[[ 2  1 10]] 2
2 4


array([[15, 12,  6,  7,  8],
       [ 2, 11,  3,  4,  5],
       [ 2,  9, 10,  1,  2]])

In [415]:
def cluster_intersection(results1, results2):
    
    size = []
    assignments = []
    for i in range(len(results1["cluster"].unique())):
        size.append(results1[results1["cluster"] == i].count()["cluster"])
        assignments.append(results1[results1["cluster"] == i][results1.columns[0:2]].values)
    
    assignments_ordered = []
    ranks = np.argsort(size)
    for j in range(len(size)):
        assignments_ordered.append(assignments[ranks[j]])
    
    size2 = []
    assignments2 = []
    for i in range(len(results2["cluster"].unique())):
        size2.append(results2[results2["cluster"] == i].count()["cluster"])
        assignments2.append(results2[results2["cluster"] == i][results2.columns[0:2]].values)
    
    assignments_ordered2 = []
    ranks2 = np.argsort(size2)
    for j in range(len(size2)):
        assignments_ordered2.append(assignments2[ranks2[j]])
    
    mat = np.zeros((len(assignments_ordered), len(assignments_ordered2)))
    
    for i in range(len(assignments_ordered)):
        for j in range(len(assignments_ordered2)):
            mat[i, j] = len(intersect2D(assignments_ordered[i], assignments_ordered2[j]))
    
    v1 = np.array(size)
    v2 = np.array(size2)
    v1 = v1[ranks]
    v2 = v2[ranks2]
    
    m1 = mat/v1[:, None]
    m2 = mat/v2[None, :]
       
    m1 = max_swap(m1)
    m2 = max_swap(m2)
    return m1, m2 

In [416]:
m1, m2 = cluster_intersection(bayesian_results,kmeans10)

[[0.03119043 0.0324034  0.03430948 0.04245365 0.05146422 0.0507711
  0.05666262 0.06376711 0.06861896 0.07399064]
 [0.02755051 0.04474871 0.03957255 0.05059275 0.06344966 0.04441476
  0.05510102 0.0637836  0.06294874 0.07113041]
 [0.0317069  0.03757225 0.03383203 0.04530772 0.04802788 0.05074804
  0.05125808 0.06715403 0.06944917 0.08245495]
 [0.0245023  0.03712098 0.03957121 0.04967841 0.04049005 0.04833078
  0.05678407 0.07540582 0.06983155 0.08508423]
 [0.02795351 0.03256339 0.03383846 0.04570644 0.05046344 0.0565936
  0.05840812 0.06478348 0.07532735 0.08552793]] 0
9 8


IndexError: index 8 is out of bounds for axis 0 with size 5

In [249]:
a

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [166]:
ind = np.unravel_index(np.argsort(a, axis=None), a.shape)
np.argsort(a.flatten())

array([5, 4, 3, 8, 7, 6, 2, 1, 0], dtype=int64)

In [3]:
import numpy as np

In [81]:
intersect2D(assignments_ordered[1], assignments_ordered[4])

array([], dtype=float64)

Clustering results with kmeans and 10 centers

In [86]:
len(intersect2D(assignments_ordered[0], assignments_ordered2[0]))

180

Gaussian mixture model with 5 components clustering

In [ ]:
twosides = get_twosides_meddra(pickle=False)
kmeans_results = pd.read_csv("../results/gaussian_mixture/gmm_10_results.csv")
kmeans_meddra = match_meddra(kmeans_results, twosides)

(kmeans_meddra.groupby(["cluster", "soc_term"]).size()/kmeans_meddra.groupby(["cluster"]).size())\
.sort_values(ascending=False)\
.sort_index(level=0, sort_remaining=False)\
.groupby("cluster").nlargest(5)